In [3]:
def get_fitness_map(fitness_scores):
    """
    This function calculates the cumulative sum of the fitness scores.
    It is used in fitness proportionate selection to determine the selection probability for each individual.
    """
    return np.cumsum(fitness_scores).tolist()

In [4]:
def select_parent(fitmap):
    """
    This function selects a parent for reproduction based on fitness proportionate selection (also known as roulette-wheel selection).
    Individuals with higher fitness have a higher chance of being selected.
    """
    idx = np.searchsorted(fitmap, np.random.uniform(0, fitmap[-1]))
    return idx

In [5]:
def create_population(pop_size, verbose):
    """
    This function creates a population of a specified size.
    Each individual in the population is an encoded genome.
    """
    return [encode_genome(verbose) for i in range(pop_size)]

In [6]:
def select_parents(fitmap):
    """
    This function selects two distinct parents from the population based on fitness scores.
    If two parents selected are the same, it will reselect the second parent until a distinct one is found.
    """
    parent_1 = select_parent(fitmap)
    parent_2 = select_parent(fitmap)
    while parent_1 == parent_2:
        parent_2 = select_parent(fitmap)
    
    return parent_1, parent_2


In [7]:
def two_point_crossover(parent1, parent2):
    flatten_index1 = next((i for i, gene in enumerate(parent1) if gene[0] == 5), None)
    flatten_index2 = next((i for i, gene in enumerate(parent2) if gene[0] == 5), None)

    parent1_pre_flatten = parent1[1:flatten_index1]
    parent2_pre_flatten = parent2[1:flatten_index2]

    conv_blocks_location1 = [i for i, gene in enumerate(parent1_pre_flatten) if gene[0] == 0]
    conv_blocks_location2 = [i for i, gene in enumerate(parent2_pre_flatten) if gene[0] == 0]
    
    if conv_blocks_location1:  # Check if the list is not empty
        crossover_point1 = random.choice(conv_blocks_location1)
    else:
        # Default value
        crossover_point1 = 0  # or any suitable default value

    if conv_blocks_location2:  # Check if the list is not empty
        crossover_point2 = random.choice(conv_blocks_location2)
    else:
        # Default value
        crossover_point2 = 0  # or any suitable default value 

    print(f"\n\nCrossver operation:\n  Crossover 1 pre_flatten: {crossover_point1}\n  Crossover 2 pre_flatten: {crossover_point2}")
    child1 = [parent1[0]] + parent1_pre_flatten[:crossover_point1] + parent2_pre_flatten[crossover_point2:]
    child2 = [parent2[0]] + parent2_pre_flatten[:crossover_point2] + parent1_pre_flatten[crossover_point1:]
    print(f"    Child 1: {child1}\n    Child 2: {child2}")

    parent1_post_flatten = parent1[flatten_index1+1:-3] # we want to capture the part of the genome from the flatten to the last dense layer both not included
    parent2_post_flatten = parent2[flatten_index2+1:-3] # last two layers must be preserved as they keep information about the compiling and fit
    
    dense_blocks_location = []
    
    dense_blocks_location1 = [i for i, gene in enumerate(parent1_post_flatten) if gene[0] == 4]
    dense_blocks_location2 = [i for i, gene in enumerate(parent2_post_flatten) if gene[0] == 4]

    if len(dense_blocks_location1) == 0:
        crossover_point1 = 0
    else:
        crossover_point1 = random.choice(dense_blocks_location1)

    if len(dense_blocks_location2) == 0:
        crossover_point2 = 0
    else:
        crossover_point2 = random.choice(dense_blocks_location2)

    print(f"\n\nCrossver operation:\n  Crossover 1 post_flatten: {crossover_point1}\n  Crossover 2 post_flatten: {crossover_point2}")
    child1 = child1 + [[5,-1]] + parent1_post_flatten[:crossover_point1] + parent2_post_flatten[crossover_point2:] + parent1[-3:]
    child2 = child2 + [[5,-1]] + parent2_post_flatten[:crossover_point2] + parent1_post_flatten[crossover_point1:] + parent2[-3:]
    
    print(f"    Child 1: {child1}\n    Child 2: {child2}")
    
    return child1, child2


def repair_offspring(offspring):

    input_dim = params['input_dim']
    input_lowest_dim = min(input_dim[0], input_dim[1])

    max_filters = offspring[0][1]
    
    

    i = 0
    while i < len(offspring):
        gene = offspring[i]
        # print(f"\nGene {i} in offstrping repair: {gene}")
        # This makes sure that the number of filters never decreases
        if gene[0] == 0:
            if gene[1] < max_filters:
                gene[1] = max_filters
            else:
                max_filters = gene[1]

        # This ensures dimension always > 1
        if gene[0] == 2:
            input_lowest_dim = input_lowest_dim / 2
            if input_lowest_dim < 1:
                # Remove this gene
                del offspring[i]
                # Readjust input_lowest_dim
                input_lowest_dim = input_lowest_dim * 2
                # Continue to the next iteration without incrementing i
                continue

        i += 1

    return offspring

In [4]:
def mutate_genome(genome, mutation_rate, mutation_type):
    """
    Mutates the given genome.
    
    Args:
        genome: List of lists, where each inner list represents a gene.
        mutation_rate: The chance of each gene getting mutated.
        mutation_type: Type of mutation, "gaussian" or "random_resetting".
        verbose: If True, prints more detailed information.
        
    Returns:
        The mutated genome.
    """
    
    for gene in genome[:-3]:
        if random.random() < mutation_rate:
            # Apply mutation based on the mutation type
            if mutation_type == 'gaussian':
                print(f"Mutating gene: {gene} in gaussian method")
                if gene[0] in [0, 4]:  # for Conv2D and Dense layers
                    gene[1] += int(np.random.normal(0, 2))
                    gene[1] = max(gene[1], 1)  # Ensure a minimum of 1 filter or neuron

            elif mutation_type == 'resetting':
                print(f"Mutating gene: {gene} in resetting method")
                if gene[0] in [0, 4]:  # for Conv2D and Dense layers
                    gene[1] = random.choice([16, 32, 64, 128, 256, 512, 1024])
    
 
    return genome


In [5]:
def generate_new_pop(pop, new_pop, mut_rate, mut_type, fitmap):
    while len(new_pop) < len(pop):
        parent1_idx, parent2_idx = select_parents(fitmap)
        print('\nParent indices:', parent1_idx, parent2_idx)
        child1, child2 = two_point_crossover(pop[parent1_idx], pop[parent2_idx])
        print(f"Confirm : Child 1: {child1}\n    Child 2: {child2}")
        child1 = mutate_genome(child1, mut_rate, random.choice(mut_type))
        child1 = repair_offspring(child1)
        new_pop.append(child1)
        if len(new_pop) < len(pop):
            child2 = mutate_genome(child2, mut_rate, random.choice(mut_type))
            child2 = repair_offspring(child2)
            new_pop.append(child2)
    return new_pop

In [ ]:
def retrieve_genomes_from_file(file_path):
    genomes = []
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            genome = []
            gene = []
            for val in row[1:]:
                if val == "-1":
                    gene.append(int(val))  # add -1 at the end of each gene
                    genome.append(gene)
                    gene = []
                else:
                    float_val = float(val)
                    if float_val.is_integer():
                        gene.append(int(float_val))  # cast to int if it is an integer
                    else:
                        gene.append(float_val)  # keep as float otherwise
            genomes.append(genome)
    return genomes

In [ ]:
def generate_population(pop_size, genome_file_path=None, verbose=0):
    new_pop = []
    if genome_file_path:  # if path is provided
        try:
            genomes_from_file = retrieve_genomes_from_file(genome_file_path)
            if genomes_from_file:  # if file has at least one genome
                print(f"{len(genomes_from_file)} genomes loaded from file")
                if len(genomes_from_file) >= pop_size:
                    new_pop = genomes_from_file[:pop_size]
                else:  # when the file contains fewer genomes than pop_size
                    new_pop = genomes_from_file + [encode_genome(verbose) for i in range(pop_size - len(genomes_from_file))]
            else:
                print("File does not contain any genome, generating new population")
                new_pop = [encode_genome(verbose) for i in range(pop_size)]
        except FileNotFoundError:
            print("File not found, generating new population")
            new_pop = [encode_genome(verbose) for i in range(pop_size)]
    else:
        print("No file provided, generating new population")
        new_pop = [encode_genome(verbose) for i in range(pop_size)]
        
        # Print all genomes in the population
    for i, genome in enumerate(new_pop, 1):
        print(f"Genome {i}: {genome}")
    return new_pop